# Museum Attendance vs City Population

This notebook reads the `museum_city_features` table (populated by the pipeline)
and visualises the linear regression model.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import text

from wikiapp.db import get_engine, get_session
from wikiapp.model import load_latest_model, summary_from_db

sns.set_theme(style="whitegrid")
engine = get_engine()

## 1. Load Data

In [ ]:
with get_session(engine) as session:
    df = pd.read_sql(
        text("SELECT museum_name, city, country, annual_visitors, population FROM museum_city_features"),
        session.connection(),
    )
print(f"{len(df)} museums in feature table")
df.head(10)

## 2. Distributions

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(df["annual_visitors"] / 1e6, bins=12, edgecolor="black", alpha=0.7)
axes[0].set_xlabel("Annual Visitors (millions)")
axes[0].set_ylabel("Count")
axes[0].set_title("Distribution of Museum Visitors")

axes[1].hist(df["population"] / 1e6, bins=12, edgecolor="black", alpha=0.7, color="orange")
axes[1].set_xlabel("City Population (millions)")
axes[1].set_ylabel("Count")
axes[1].set_title("Distribution of City Populations")

plt.tight_layout()
plt.show()

## 3. Regression

In [ ]:
model, version = load_latest_model(engine)
metrics = summary_from_db(engine)
print(f"Model version: {version}")
print(f"R\u00b2 = {metrics['r2']:.4f}  |  RMSE = {metrics['rmse']:,.0f}  |  MAE = {metrics['mae']:,.0f}")
coef, intercept = float(model.coef_[0]), float(model.intercept_)
print(f"visitors = {coef:.4f} * population + {intercept:.0f}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

ax.scatter(df["population"] / 1e6, df["annual_visitors"] / 1e6,
           s=80, alpha=0.7, edgecolors="black", linewidths=0.5, zorder=5)

for _, row in df.iterrows():
    ax.annotate(row["museum_name"],
                (row["population"] / 1e6, row["annual_visitors"] / 1e6),
                fontsize=7, alpha=0.8, xytext=(5, 5), textcoords="offset points")

x_range = np.linspace(df["population"].min(), df["population"].max(), 100)
y_range = model.predict(x_range.reshape(-1, 1))
ax.plot(x_range / 1e6, y_range / 1e6, color="red", linewidth=2,
        label=f"y = {coef:.4f}x + {intercept:.0f}\nR\u00b2 = {metrics['r2']:.4f}")

ax.set_xlabel("City Population (millions)", fontsize=12)
ax.set_ylabel("Annual Museum Visitors (millions)", fontsize=12)
ax.set_title("Museum Visitors vs City Population \u2014 Linear Regression", fontsize=14)
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

## 4. Residual Analysis

In [ ]:
df["predicted"] = model.predict(df[["population"]].to_numpy(dtype=float))
df["residual"] = df["annual_visitors"] - df["predicted"]

fig, ax = plt.subplots(figsize=(10, 5))
colors = ["green" if r > 0 else "red" for r in df["residual"]]
ax.barh(df["museum_name"], df["residual"] / 1e6, color=colors, alpha=0.7)
ax.set_xlabel("Residual (millions of visitors)")
ax.set_title("Residuals: Actual \u2212 Predicted Visitors")
ax.axvline(0, color="black", linewidth=0.8)
plt.tight_layout()
plt.show()

## Interpretation

- **Positive residuals** (green): museums that outperform their city\u2019s population prediction \u2014 driven by international tourism, free admission, or iconic status.
- **Negative residuals** (red): museums below prediction \u2014 possibly in cities with many competing museums or less international draw.
- The **low R\u00b2** is expected: city population alone is a weak predictor. Tourist infrastructure, museum type, pricing, and reputation matter more.

### Next Steps
- Add features (GDP, tourism arrivals, museum type, free-admission flag)
- Try log-log regression or polynomial features
- Aggregate museums by city for a city-level analysis
- Use cross-validation with a larger dataset